# Install the PIC-SURE-HPDS client library

In [ ]:
import sys
import json

#!{sys.executable} -m pip uninstall git+https://github.com/hms-dbmi/pic-sure-python-client.git    
#!{sys.executable} -m pip uninstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade git+https://github.com/hms-dbmi/pic-sure-python-client.git    
!{sys.executable} -m pip install --upgrade git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib

# We are connecting directly to the HPDS resource, bypassing all security. This should never be made possible in an instance holding private data. We do this by creating a Resource Adaptor for HDPS that bypasses PIC-SURE and get the only resource it has access to. 

# An HPDS instance should always be hosted behind a PIC-SURE API and PIC-SURE Auth Micro-App instance if it has any privacy concern. 

# This is only for the example


In [ ]:
from pandas import pandas as pd
import PicSureHpdsLib
adapter = PicSureHpdsLib.BypassAdapter("http://pic-sure-hpds-1kgenome:8080/PIC-SURE")
resource = adapter.useResource()

# To see what is possible using this resource adaptor let's look at the interactive documentation.

In [ ]:
resource.help()

# We have a couple things we can do, we can access the data dictionary, or we can run a query. First let's see the annotations that were loaded from the INFO column of the source VCF. To do this we search the data dictionary for everything(empty string)

In [ ]:
data_dictionary = resource.dictionary().find("")
data_dictionary.DataFrame()

# Now that we have seen what kind of data is indexed in the dictionary, let's figure out how to query it. We strive to document each function of the client library with interactive help. This is the help for the query() object

In [ ]:
resource.query().help()

# In order to run a query, we generally want to filter the patients in scope. We can use the data dictionary to create filters for this. For example, if we want to filter by a categorical variable such as the source call set variable, we just add a filter to our query and get a count of patients by calling getCount() on the query.

In [ ]:
CS_SVA_umary = resource.query()
CS_SVA_umary.filter().add("VT",["SNP"])
CS_SVA_umary.getCount()

# If we want to filter by a numerical variable such as the estimated allele frequency we can add a numerical range filter then call getCount()

In [ ]:
AF_001 = resource.query()
AF_001.filter().add("AF", min=.000, max=.001)
AF_001.getCount()

# Something to be careful of when using numerical range filtering is that it is really easy to end up building a filter that includes all patients. Everyone has at least 1 rare variant, so we get all patients back. This particular filter involves 141,362 different variants. The total size of our dataset is 2504 patients, and at least 1 patient has every one of the variants in our dataset.
# To make this a more useful(and faster) query we should add additional filters, let's combine the 2 previous filters. This will only include variants that have source call set value of SVA_umary and an estimated allele frequency in the general population of less than .1%

In [ ]:
CS_SVA_umary = resource.query()
CS_SVA_umary.filter().add("CS",["SVA_umary"])
CS_SVA_umary.filter().add("AF", min=.000, max=.001)
CS_SVA_umary.getCount()

# We can also query on a specific variant of interest and specific zygosities for that variant. For example, let's see how many subjects have homozygous G->A for offset 19073032 on chromosome 14 

In [ ]:
homozygous_G_A = resource.query()
homozygous_G_A.filter().add("14,19073032,G,A",["1/1"])
homozygous_G_A.getCount()

# We can also see how many subjects have either homozygous or heterozygous G->A for offset 19073032 on chromosome 14 

In [ ]:
any_G_A = resource.query()
any_G_A.filter().add("14,19073032,G,A",["1/1", "0/1"])
any_G_A.getCount()

# We can also find patients who have both homozygous 14,19007199,C,T and heterozygous 14,19073032,G,A

In [ ]:
two_variants_and = resource.query()
two_variants_and.filter().add("14,19007199,C,T",["1/1"])
two_variants_and.filter().add("14,19073032,G,A",["0/1"])
two_variants_and.getCount()

# If we are interested in the ids for those patients, we can get the results dataframe for the query also

In [ ]:
two_variants_and.getResultsDataFrame()

# If we had phenotype data we could probably come up with a better use-case here, but we will save that for another example. Let's create two subsets and find the counts for each of those subsets for 2600 different variants.
# Subset A are people who are homozygous-reference for 14,19073032,G,A
# Subset B are people who have any copies of the variant

In [ ]:
chr14_19073032_G_only = resource.query()
chr14_19073032_G_only.filter().add("14,19073032,G,A",["0/0"])
chr14_19073032_G_only.getCount()

In [ ]:
chr14_19073032_G_A = resource.query()
chr14_19073032_G_A.filter().add("14,19073032,G,A",["1/1","0/1"])
chr14_19073032_G_A.getCount()

# Now let's see how many subjects in each subset have each variant in a list of 2600 variants of our choosing


In [ ]:
variants = []
file = open('2600variants.txt', 'r')

for variant in file:
    variants.append(variant.replace('\n', ''))
    
chr14_19073032_G_only.crosscounts().clear()
chr14_19073032_G_only.crosscounts().add(variants)
chr14_19073032_G_only_counts = chr14_19073032_G_only.getCrossCounts()

chr14_19073032_G_A.crosscounts().clear()
chr14_19073032_G_A.crosscounts().add(variants)
chr14_19073032_G_A_counts = chr14_19073032_G_A.getCrossCounts()

In [ ]:
countsDataframe = pd.DataFrame([chr14_19073032_G_only_counts,chr14_19073032_G_A_counts])
countsDataframe

# Those counts seem a bit disproportionate to me, maybe there is a correlation between 14,19073032,G,A and 14,19000141,T,C but I'll leave that up to you to figure out.


In [ ]:
countsDataframe['14,19000141,T,C']


# You can also run this same query for the allVariants.txt file, which has 360k variants. It will take only a few minutes if you do.

# Please also check out our phenotype example here https://github.com/hms-dbmi/pic-sure-hpds-phenotype-load-example 